In [10]:
import csv
import codecs
import pprint
import re
import xml.etree.cElementTree as ET

import cerberus

import schema

DATA_DIR = 'Data/'
CSV_DIR = 'CSVs/'
OSM_PATH = "SLO.osm"
# OSM_PATH = "SLO_shortened.osm"
# OSM_PATH = "example.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def handle_tags(element, tags):
    for tag in element.iter('tag'):
        d = {'id': element.attrib['id'],
             'value': tag.attrib['v']}
             
        m = PROBLEMCHARS.search(tag.attrib['k'])
        if not m:
            m = LOWER_COLON.search(tag.attrib['k'])
            if m:
                rgx = r'(.*?):(.*)'
                m = re.match(rgx, tag.attrib['k'])
                
                d['key'] = m.group(2)
                d['type'] = m.group(1)
            else:
                d['key'] = tag.attrib['k']
                d['type'] = 'regular'
        tags.append(d)


def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        node_attribs = dict((k, element.attrib[k]) for k in NODE_FIELDS)

    elif element.tag == 'way':
        way_attribs = dict((k, element.attrib[k]) for k in WAY_FIELDS)
        
        for i, tag in enumerate(element.iter('nd')):
            d = {'id': element.attrib['id'],
                 'node_id': tag.attrib['ref'],
                 'position': i}
            way_nodes.append(d)
        
    handle_tags(element, tags)
        
    
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
        

# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.items())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
#             k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.items()
                # Okay I admit my understanding of encoding is a little fuzzy, but this should work
            k: (v.encode('utf-8') if isinstance(v, str) else v) for k, v in row.items()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

In [11]:
# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(CSV_DIR + NODES_PATH, 'w') as nodes_file, \
         codecs.open(CSV_DIR + NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(CSV_DIR + WAYS_PATH, 'w') as ways_file, \
         codecs.open(CSV_DIR + WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(CSV_DIR + WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for i, element in enumerate(get_element(file_in, tags=('node', 'way'))):
            el = shape_element(element)
            if el:
                # Only gonna validate every 100th
                if validate is True and i % 100 == 0:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(DATA_DIR + OSM_PATH, validate=False)